In [ ]:
!pip install yt-dlp
!pip install gradio
!pip install gtts

In [ ]:
import os
DEVICE = 'cuda' if 'COLAB_GPU' in os.environ else 'cpu'

In [ ]:
from yt_dlp import YoutubeDL
import glob

def get_transcript(url):
    with YoutubeDL({"skip_download":True, "writeautomaticsub":True, "subtitleslangs":["en", "en-us", "en-us", "en-uk", "en-in", "en-es", "en-fr"]}) as ydl:
        if ydl.download(url):
            print("-----------------------------------------------------------------------------------------------------------LINK FAILED")
            return None, None
        else:
            # print(ydl.extract_info(url))
            op = [f for f in os.listdir() if f.startswith(os.path.splitext(ydl.prepare_filename(ydl.extract_info(url)))[0]) and f.endswith('.vtt')]
            if op:
              return ydl.extract_info(url)['title'], op[0]
            else:
              print("-----------------------------------------------------------------------------------------------------------file download FAILED")
              return None, None

In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer
import torch, math

PROMPT_LEN, MAX_N_TOKEN = 1024, 1024

class SUMMARY:
    def __init__(self):
        torch.cuda.empty_cache()
        self.model_ckpt = "facebook/bart-large-cnn"
        self.device = DEVICE
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_ckpt)
        self.model = BartForConditionalGeneration.from_pretrained(self.model_ckpt).to(
            self.device
        )

    def summarize(self, text, first=True):
        input_ids = self.tokenizer.encode(text, return_tensors="pt").to(self.device)
        if input_ids.shape[1] <= PROMPT_LEN and not first:
            summary = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
            print("Summary is ->>>>>>>>>>>>>>>>>>>>>>>>>\n" + summary)
            return summary

        n_split = math.ceil(input_ids.shape[1] / MAX_N_TOKEN)
        splits = [
            input_ids[:, MAX_N_TOKEN * i : MAX_N_TOKEN * i + MAX_N_TOKEN]
            for i in range(0, n_split)
        ]
        summarized_splits = [
            self.model.generate(split, max_length=PROMPT_LEN) for split in splits
        ]
        decoded_summary_splits = [
            self.tokenizer.decode(s[0], skip_special_tokens=True)
            for s in summarized_splits
        ]
        summary = " ".join(decoded_summary_splits)
        print(
            "Intermediate Summaries ->>>>>>>>>>>>>>>>>>>>>>>>>\n",
            decoded_summary_splits,
        )
        return self.summarize(summary, first=False)

summarizer=SUMMARY()

In [ ]:
def video_summarize(url):
  title, vttfile=get_transcript(url)
  if vttfile is None:
    return "SOME ISSUE WITH VIDEO LINK OR DOWNLOAINDING VIDEO CONTENTS"
  print("text file expected", vttfile+'.txt')
  os.system(f"cat \"{vttfile}\" | grep : -v | awk '!seen[$0]++' > \"{vttfile}.txt\"")
  tscript = open(f'{vttfile}.txt').read()
  return title, summarizer.summarize(tscript)

In [ ]:
import gradio as gr
from gtts import gTTS

def text_to_speech(text):
    audio = gTTS(text=text, lang="en", tld='co.in',  slow=False)
    audio.save("bot.mp3")
    return "bot.mp3"

with gr.Blocks(title="Youtube Video Summarizer") as demo:
    inputlink = gr.Textbox(label="Insert an youtube link here", autofocus=True)
    btn = gr.Button("Submit")
    outputtext = gr.Textbox(label="Summary")
    opaudio = gr.Audio(label="Audio Summary", source="microphone", type="filepath", autoplay=True)
    clear = gr.ClearButton([outputtext, inputlink, opaudio])

    def respond(link):
        title, summary = video_summarize(link.strip())
        return title + '\n\n' + summary, text_to_speech(summary)

    btn.click(lambda :None, None, inputlink)
    btn.click(lambda :None, None, outputtext)
    btn.click(lambda :None, None, opaudio)
    btn.click(respond, [inputlink], [outputtext, opaudio])

    inputlink.submit(lambda :None, None, inputlink)
    inputlink.submit(lambda :None, None, outputtext)
    inputlink.submit(lambda :None, None, opaudio)
    inputlink.submit(respond, [inputlink], [outputtext, opaudio])

demo.queue(concurrency_count=1)
demo.launch(debug=True, share=True).queue()
pass

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://494d4a51e091af7fed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[youtube] Extracting URL: https://www.youtube.com/watch?v=DHaxfzNT1KA
[youtube] DHaxfzNT1KA: Downloading webpage


[youtube] DHaxfzNT1KA: Downloading ios player API JSON
[youtube] DHaxfzNT1KA: Downloading android player API JSON
[youtube] DHaxfzNT1KA: Downloading iframe API JS
[youtube] DHaxfzNT1KA: Downloading player da7c2a60
[youtube] DHaxfzNT1KA: Downloading web player API JSON
[youtube] DHaxfzNT1KA: Downloading m3u8 information


[youtube] DHaxfzNT1KA: Downloading initial data API JSON
[info] DHaxfzNT1KA: Downloading subtitles: en, en-US
[info] DHaxfzNT1KA: Downloading 1 format(s): 616+251
[info] Writing video subtitles to: Gravitational wave discovery leads to greater understanding of the fabric of our universe [DHaxfzNT1KA].en.vtt
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1
[download] Destination: Gravitational wave discovery leads to greater understanding of the fabric of our universe [DHaxfzNT1KA].en.vtt
[download] 100% of    9.58KiB in 00:00:00 at 168.25KiB/s             
[info] Writing video subtitles to: Gravitational wave discovery leads to greater understanding of the fabric of our universe [DHaxfzNT1KA].en-US.vtt
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1
[download] Destination: Gravitational wave discovery leads to greater understanding of the fabric of our universe [DHaxfzNT1KA].en-US.vtt
[download] 100% of    8.40KiB in 00:00:00 at 225.07KiB/s     

[youtube] DHaxfzNT1KA: Downloading ios player API JSON
[youtube] DHaxfzNT1KA: Downloading android player API JSON
[youtube] DHaxfzNT1KA: Downloading iframe API JS
[youtube] DHaxfzNT1KA: Downloading web player API JSON
[youtube] DHaxfzNT1KA: Downloading m3u8 information


...

[youtube] FfgT6zx4k3Q: Downloading ios player API JSON
[youtube] FfgT6zx4k3Q: Downloading android player API JSON
[youtube] FfgT6zx4k3Q: Downloading iframe API JS
[youtube] FfgT6zx4k3Q: Downloading web player API JSON
[youtube] FfgT6zx4k3Q: Downloading m3u8 information


[youtube] FfgT6zx4k3Q: Downloading initial data API JSON
[info] FfgT6zx4k3Q: Downloading subtitles: en, en-es, en-fr
[info] FfgT6zx4k3Q: Downloading 1 format(s): 303+251
Deleting existing file Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en.vtt
[info] Writing video subtitles to: Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en.vtt
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1
[download] Destination: Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en.vtt
[download] 100% of   11.57KiB in 00:00:00 at 248.48KiB/s             
Deleting existing file Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en-es.vtt
[info] Writing video subtitles to: Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en-es.vtt
[download] Destination: Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en-es.vtt
[download] 100% of   11.20KiB in 00:00:00 at 253.86KiB/s
Deleting existing

[youtube] FfgT6zx4k3Q: Downloading ios player API JSON
[youtube] FfgT6zx4k3Q: Downloading android player API JSON
[youtube] FfgT6zx4k3Q: Downloading iframe API JS
[youtube] FfgT6zx4k3Q: Downloading web player API JSON
[youtube] FfgT6zx4k3Q: Downloading m3u8 information


[youtube] FfgT6zx4k3Q: Downloading initial data API JSON
[info] FfgT6zx4k3Q: Downloading subtitles: en, en-es, en-fr
[info] FfgT6zx4k3Q: Downloading 1 format(s): 303+251
Deleting existing file Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en.vtt
[info] Writing video subtitles to: Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en.vtt
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1
[download] Destination: Could Your Phone Hurt You？ Electromagnetic Pollution [FfgT6zx4k3Q].en.vtt
[download] fragment not found; Skipping fragment 1 ...


ERROR: The downloaded file is empty
ERROR: Unable to download video subtitles for 'en': ERROR: The downloaded file is empty
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/YoutubeDL.py", line 4147, in _write_subtitles
    self.dl(sub_filename, sub_copy, subtitle=True)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/YoutubeDL.py", line 3084, in dl
    return fd.download(name, new_info, subtitle)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/downloader/common.py", line 461, in download
    ret = self.real_download(filename, info_dict)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/downloader/hls.py", line 372, in real_download
    self.download_and_append_fragments(
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/downloader/fragment.py", line 542, in download_and_append_fragments
    return self._finish_frag_download(ctx, info_dict)
  File "/usr/local/lib/python3.10/dist-packages/yt_dlp/downloader/fragment.py", line